![](img/572_banner.png)

# Lab 3: Convolutional Neural Networks

**Tomas Beuzen, January 2021**

It's quiz week and we've been hitting it hard in lectures so I've got a nice short lab for you this week. I'll be getting you to review basic CNN concepts and to implement a CNN from scratch in PyTorch - let's do it! Beep boop beep boop!

![](img/robot.png)

## Table of Contents
<hr>

<div class="toc"><ul class="toc-item"><li><span><a href="#Instructions" data-toc-modified-id="Instructions-2">Instructions</a></span></li><li><span><a href="#Imports" data-toc-modified-id="Imports-3">Imports</a></span></li><li><span><a href="#Exercise-1:-Filters-and-Convolutions" data-toc-modified-id="Exercise-1:-Filters-and-Convolutions-4">Exercise 1: Filters and Convolutions</a></span></li><li><span><a href="#Exercise-2:-CNNs-and-Image-Permutations" data-toc-modified-id="Exercise-2:-CNNs-and-Image-Permutations-5">Exercise 2: CNNs and Image Permutations</a></span></li><li><span><a href="#Exercise-3:-Bitmoji-CNN" data-toc-modified-id="Exercise-3:-Bitmoji-CNN-6">Exercise 3: Bitmoji CNN</a></span></li><li><span><a href="#(Optional)-Exercise-4:-Neural-Network-and-Backpropagation-&quot;By-Hand&quot;" data-toc-modified-id="(Optional)-Exercise-4:-Neural-Network-and-Backpropagation-&quot;By-Hand&quot;-7">(Optional) Exercise 4: Neural Network and Backpropagation "By Hand"</a></span></li><li><span><a href="#Submit-to-Canvas-and-GitHub" data-toc-modified-id="Submit-to-Canvas-and-GitHub-8">Submit to Canvas and GitHub</a></span></li></ul></div>

## Instructions
<hr>

rubric={mechanics:3}

**Link to your GitHub repository:**

You will receive marks for correctly submitting this assignment. To submit this assignment you should:

1. Push your assignment to your GitHub repository!
2. Provide a link to your repository in the space provided above.
2. Upload a HTML render of your assignment to Canvas. The last cell of this notebook will help you do that.
3. Be sure to follow the [General Lab Instructions](https://ubc-mds.github.io/resources_pages/general_lab_instructions/). You can view a description of the different rubrics used for grading in MDS [here](https://github.com/UBC-MDS/public/tree/master/rubric).

Here's a break down of the required and optional exercises in this lab:

|         | Number of Exercises | Points |
|:-------:|:-------------------:|:------:|
| Required| 9 | 30 |
| Optional| 1  | 1 |

## Imports
<hr>

In [ ]:
import numpy as np
import pandas as pd
import torch
from torch import nn, optim
from torchvision import datasets, transforms, utils
from torchsummary import summary
import matplotlib.pyplot as plt
plt.style.use('ggplot')
plt.rcParams.update({'font.size': 16, 'axes.labelweight': 'bold', 'axes.grid': False})
from canvasutils.submit import submit, convert_notebook

## Exercise 1: Filters and Convolutions
<hr>

Here's an image of my dog Evie (so cute I know!):

In [ ]:
image = torch.from_numpy(plt.imread("img/evie.png"))
plt.imshow(image, cmap='gray')
plt.axis('off');

For each of the filters given below, convolve the image of Evie with the given filter/kernel and briefly discuss why the results look the way they do. You'll need to:
1. Created a `Conv2D` layer with PyTorch.
2. Manually change the kernel weights. Weights in a `Conv2D` layer are 4D tensors (the 4 dimensions are: `[num_images=1, num_channels=1, kernel_rows, kernel_cols]`) and changing the `Conv2D` layer weights (I've given example code defining such a 4D tensor below). Functions that will help you create tensors: `torch.ones()`, `torch.zeros()`, `torch.full()`, etc. (remember, we have much of the same functionality as we did in `NumPy`!).
3. Use the provided code to plot the original and convolved images.
4. Explain the result in one sentence.

I've provided an example below to get you started. You can assume the default `stride` and `padding` in the `Conv2D` layer.

>The pedagogical goal here is to help you better understand what filters/kernels actually are and how they help us identify useful structure (like lines, curves, shapes, etc.) in images.

In [ ]:
def plot_convolution(image, conv_layer):
    """
    Convolve kernel over image and plot.

    Parameters
    ----------
    image : torch.Tensor
        Image to filter with kernel.
    conv_layer : function
        A PyTorch Conv2D layer to apply to image.
    
    Returns
    -------
    matplotlib.image
    """

    conv_image = conv_layer(image[None, None, :]).detach().squeeze()
    fig, (ax1, ax2) = plt.subplots(figsize=(8, 4), ncols=2)
    ax1.imshow(image, cmap='gray'); ax1.axis('off'); ax1.set_title("Original")
    ax2.imshow(conv_image, cmap='gray'); ax2.axis('off'); ax2.set_title("Filtered")
    plt.tight_layout()

**Example:**

The kernel is a row vector of ten 0.1's, shape (1, 10):

$$kernel = \begin{bmatrix} 0.1 & 0.1 & 0.1 & 0.1 & 0.1 & 0.1 & 0.1 & 0.1 & 0.1 & 0.1 \end{bmatrix}$$

In [ ]:
conv_layer = torch.nn.Conv2d(1, 1, kernel_size=(1, 10))
kernel = torch.full((1, 1, 1, 10), 0.1)
conv_layer.weight[:] = kernel
plot_convolution(image, conv_layer)

**Example answer:**

The filter is a horizontal bar of $0.1$s. Therefore I would expect a blurring in the horizontal direction, meaning the _vertical_ edges get blurred (because these are the ones that change rapidly in the horizontal direction). This seems to be happening in the result. 

### 1.1
rubric={accuracy:2}

The kernel is a column vector of ten 0.1's, shape (10, 1):

$$kernel = \begin{bmatrix} 0.1 \\ 0.1 \\ 0.1 \\ 0.1 \\ 0.1 \\ 0.1 \\ 0.1 \\ 0.1 \\ 0.1 \\ 0.1 \end{bmatrix}$$

In [ ]:
# Your answer goes here.

### 1.2
rubric={accuracy:2}

The kernel is a matrix of 0's but with a 1 in the centre, shape (5, 5):

$$kernel = \begin{bmatrix} 0 & 0 & 0 & 0 & 0 \\ 0 & 0 & 0 & 0 & 0 \\ 0 & 0 & 1 & 0 & 0 \\ 0 & 0 & 0 & 0 & 0 \\ 0 & 0 & 0 & 0 & 0 \\ \end{bmatrix}$$

In [ ]:
# Your answer goes here.

### 1.3
rubric={accuracy:2}

The kernel is a matrix of 0.01's, shape (10, 10):

$$kernel = \begin{bmatrix} 0.01 & 0.01 & 0.01 & 0.01 & 0.01 & 0.01 & 0.01 & 0.01 & 0.01 & 0.01 & \\ 0.01 & 0.01 & 0.01 & 0.01 & 0.01 & 0.01 & 0.01 & 0.01 & 0.01 & 0.01 & \\ 0.01 & 0.01 & 0.01 & 0.01 & 0.01 & 0.01 & 0.01 & 0.01 & 0.01 & 0.01 & \\ 0.01 & 0.01 & 0.01 & 0.01 & 0.01 & 0.01 & 0.01 & 0.01 & 0.01 & 0.01 & \\ 0.01 & 0.01 & 0.01 & 0.01 & 0.01 & 0.01 & 0.01 & 0.01 & 0.01 & 0.01 & \\ 0.01 & 0.01 & 0.01 & 0.01 & 0.01 & 0.01 & 0.01 & 0.01 & 0.01 & 0.01 & \\ 0.01 & 0.01 & 0.01 & 0.01 & 0.01 & 0.01 & 0.01 & 0.01 & 0.01 & 0.01 & \\ 0.01 & 0.01 & 0.01 & 0.01 & 0.01 & 0.01 & 0.01 & 0.01 & 0.01 & 0.01 & \\ 0.01 & 0.01 & 0.01 & 0.01 & 0.01 & 0.01 & 0.01 & 0.01 & 0.01 & 0.01 & \\ 0.01 & 0.01 & 0.01 & 0.01 & 0.01 & 0.01 & 0.01 & 0.01 & 0.01 & 0.01 \end{bmatrix}$$

In [ ]:
# Your answer goes here.

### 1.4
rubric={accuracy:2}

The kernel is a matrix of -0.125's, shape (3, 3):

$$kernel = \begin{bmatrix} -0.125 & -0.125 & -0.125 \\ -0.125 & -0.125 & -0.125 \\ -0.125 & -0.125 & -0.125 \end{bmatrix}$$

In [ ]:
# Your answer goes here.

## Exercise 2: CNNs and Image Permutations
<hr>

Below is some code that trains a CNN on the classic [MNIST digits dataset](https://en.wikipedia.org/wiki/MNIST_database). This dataset contains 28 x 28 pixel images of hand written digits. Run through the code, it may take a few minutes to run the code, our training dataset has 60,000 samples and our validation dataset has 10,000.

In [ ]:
BATCH_SIZE = 256

# Download data
transform = transforms.Compose([transforms.ToTensor()])
trainset = datasets.MNIST('data/', download=True, train=True, transform=transform)
validset = datasets.MNIST('data/', download=True, train=False, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=BATCH_SIZE, shuffle=True)
validloader = torch.utils.data.DataLoader(validset, batch_size=BATCH_SIZE, shuffle=True)

# Sample plot
X, y = next(iter(trainloader))
plt.imshow(X[0, 0, :, :], cmap="gray")
plt.title(f"Number: {y[0].item()}");

In [ ]:
class MNIST_classifier(nn.Module):
    def __init__(self):
        super().__init__()
        self.main = nn.Sequential(
            nn.Conv2d(1, 16, (5, 5)),
            nn.ReLU(),
            nn.MaxPool2d((2,2)),
            nn.Dropout(0.2),
            nn.Flatten(),
            nn.Linear(2304, 128),
            nn.ReLU(),
            nn.Linear(128, 10)
        )
        
    def forward(self, x):
        out = self.main(x)
        return out

In [ ]:
def trainer(model, criterion, optimizer, trainloader, validloader, epochs=5, verbose=True):
    """Simple training wrapper for PyTorch network."""
    
    train_loss, valid_loss, valid_accuracy = [], [], []
    for epoch in range(epochs):  # for each epoch
        train_batch_loss = 0
        valid_batch_loss = 0
        valid_batch_acc = 0
        
        # Training
        for X, y in trainloader:
            optimizer.zero_grad()       # Zero all the gradients w.r.t. parameters
            y_hat = model(X)            # Forward pass to get output
            loss = criterion(y_hat, y)  # Calculate loss based on output
            loss.backward()             # Calculate gradients w.r.t. parameters
            optimizer.step()            # Update parameters
            train_batch_loss += loss.item()  # Add loss for this batch to running total
        train_loss.append(train_batch_loss / len(trainloader))
        
        # Validation
        with torch.no_grad():  # this stops pytorch doing computational graph stuff under-the-hood and saves memory and time
            for X, y in validloader:
                y_hat = model(X)
                _, y_hat_labels = torch.softmax(y_hat, dim=1).topk(1, dim=1)
                loss = criterion(y_hat, y)
                valid_batch_loss += loss.item()
                valid_batch_acc += (y_hat_labels.squeeze() == y).type(torch.float32).mean().item()
        valid_loss.append(valid_batch_loss / len(validloader))
        valid_accuracy.append(valid_batch_acc / len(validloader))  # accuracy
        
        # Print progress
        if verbose:
            print(f"Epoch {epoch + 1}:",
                  f"Train Loss: {train_loss[-1]:.3f}.",
                  f"Valid Loss: {valid_loss[-1]:.3f}.",
                  f"Valid Accuracy: {valid_accuracy[-1]:.2f}.")
    
    results = {"train_loss": train_loss,
               "valid_loss": valid_loss,
               "valid_accuracy": valid_accuracy}
    return results

In [ ]:
# Training may take a few minutes, we have a lot of data (60,000 training samples)
model = MNIST_classifier()
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters())
results = trainer(model, criterion, optimizer, trainloader, validloader)

We probably got pretty good accuracy there! Now, answer the questions below:

### 2.1
rubric={accuracy:1}

How many parameters does the model have?

In [ ]:
# Your answer goes here.

### 2.2
rubric={reasoning:1}

In Lecture 6 I talked about how, when doing image classification with fully connected neural networks, the order of the pixels in the flattened image we feed into the network doesn't matter. In contrast, CNNs try to use the structure in the data to make predictions. Let's do an experiment and vertically flip all our MNIST training images like this:

In [ ]:
transform = transforms.Compose([transforms.RandomVerticalFlip(p=1), transforms.ToTensor()])
trainset = datasets.MNIST('data/', download=True, train=True, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=BATCH_SIZE, shuffle=True)

# Sample plot
X, y = next(iter(trainloader))
plt.imshow(X[0, 0, :, :], cmap="gray")
plt.title(f"Number: {y[0].item()}");

We've only flipped the training images, not the validation images. What do you think would happen to the validation scores if we train our network on these new flipped images?

>Your answer goes here.



### 2.3
rubric={reasoning:2}

Re-train your network using the new `trainloader` of flipped images we defined above to test your answer to 2.2. Are the results what you expected? Why/why not?

In [ ]:
# Your answer goes here.

## Exercise 3: Bitmoji CNN
<hr>

rubric={accuracy:15}

In the student repository is a folder (`lectures/data/bitmoji_rgb`) containing **coloured** versions of the "Tom" and "Not Tom" Bitmoji images we saw in Lecture 5. Make sure you clone that repo and then copy the folder `bitmoji_rgb` to this directory (or feel free to change the path below to point to the copy in your cloned student repo). We have 857 images of both "`tom`" and "`not_tom`" for training (1714 images total), and 200 images of both "`tom`" and "`not_tom`" for validation (400 images total). We will resize the images to be 64 x 64 pixels.

Your task here is to simply create and train a CNN on this data that classifies a given image as "`tom`" or "`not_tom`" (i.e., this is binary classification). You can create any architecture you wish, but you must show me you have at least 70% accuracy on the validation data set after training your CNN (even with a very simple CNN I got >80% with just 20 epochs). I used a combination of the following layers but you can use whatever you wish:
- `torch.nn.Conv2d()`
- `torch.nn.ReLU()`
- `torch.nn.MaxPool2d()`
- `torch.nn.Dropout()`
- `torch.nn.Flatten()`
- `torch.nn.Linear()`

I have prepared the training and validation loaders for you below:

In [ ]:
TRAIN_DIR = "bitmoji_rgb/train/"
VALID_DIR = "bitmoji_rgb/valid/"
IMAGE_SIZE = 64
BATCH_SIZE = 64

# Load data and create datalaoders
data_transforms = transforms.Compose([transforms.Resize(IMAGE_SIZE), transforms.ToTensor()])
train_dataset = datasets.ImageFolder(root=TRAIN_DIR, transform=data_transforms)
trainloader = torch.utils.data.DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
valid_dataset = datasets.ImageFolder(root=VALID_DIR, transform=data_transforms)
validloader = torch.utils.data.DataLoader(valid_dataset, batch_size=BATCH_SIZE, shuffle=True)

# Plot samples (don't worry too much about this code)
sample_batch = next(iter(trainloader))
plt.figure(figsize=(10, 8)); plt.axis("off"); plt.title("Sample Training Images")
plt.imshow(np.transpose(utils.make_grid(sample_batch[0], padding=1, normalize=True),(1,2,0)));

In [ ]:
# Your answer goes here.

In [ ]:
# Here's some code you can use to plot your model's predictions for a random image from the validation set
# Depending on how you set your model up you may have to change some things to make dimensions work
img = next(iter(trainloader))[0][0]
y_prob = torch.sigmoid(model(img.unsqueeze(0)))
y_class = int(y_prob > 0.5)
plt.imshow(img.permute(1, 2, 0))
plt.axis("off")
plt.title(f"{['Not Tom', 'Tom'][y_class]} (P={y_prob.item():.2f})", pad=10);

## (Optional) Exercise 4: Neural Network and Backpropagation "By Hand"
<hr>

rubric={accuracy:1}

From scratch using only `NumPy` and `SciPy`, implement a one-hidden-layer neural network for regression using ReLUs. You can ignore the fact that the ReLU function is not differentiable when its input equals zero (it's very unlikely that we'll ever get a value of exactly 0 in our network). You're pretty much on your own for this question - it's a real test of how good your coding skills are and your knowledge of how neural networks actually work. You'll need to code up the gradients and implement backpropagation yourself. **This is a lot of work for one mark, use your time wisely**. 

Feel free to use any dataset you like to test your implementation. You can borrow the "non-linear regression" dataset I used in [Lecture 4](https://pages.github.ubc.ca/MDS-2020-21/DSCI_572_sup-learn-2_students/lectures/lecture4_pytorch-neural-networks-pt1.html#non-linear-regression-with-a-neural-network) if you like:

```python
np.random.seed(2020)
X = np.sort(np.random.randn(500))
y = X ** 2 + 15 * np.sin(X) ** 3
```

> Note: there are probably a lot of resources out there where people give their "raw" neural network implementations. If you're going to do this and you consult any sources, make sure you cite them.

In [ ]:
# Your answer goes here.

## Submit to Canvas and GitHub
<hr>

When you are ready to submit your assignment do the following:
1. Run all cells in your notebook to make sure there are no errors by doing `Kernel -> Restart Kernel and Run All Cells...`
2. Save your notebook.
3. Convert your notebook to `.html` format using the `convert_notebook()` function below or by `File -> Export Notebook As... -> Export Notebook to HTML`
4. Run the code `submit()` below to go through an interactive submission process to Canvas.
5. Finally, push all your work to GitHub (including the rendered html file).

In [ ]:
# convert_notebook("lab3.ipynb", "html")  # save your notebook, then uncomment and run when you want to convert to html

In [ ]:
# submit(course_code=59090)  # uncomment and run when ready to submit to Canvas